In [64]:
from google import genai
from google.genai import types

# ✅ LangChain memory integration
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import AIMessage, HumanMessage

# 🔐 Your API key
GOOGLE_API_KEY = "Enter your key here"
# you can get a key from this link gien below
# https://aistudio.google.com/apikey



# 🧠 Instruction for the bot
instruction = "You are a helpful assistant. Answer factual and general knowledge questions clearly."

# Initialize Gemini client directly
client = genai.Client(api_key=GOOGLE_API_KEY)

# 📚 Optional Wikipedia lookup tool
def wikipedia_lookup(query: str) -> str:
    import wikipedia
    try:
        return wikipedia.summary(query, sentences=3)
    except Exception as e:
        return f"Could not fetch info: {e}"

# Initialize LangChain memory to store chat messages
memory = ConversationBufferMemory(return_messages=True)

# Create Gemini chat session
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[wikipedia_lookup]
    )
)

# ✅ Main Q&A loop
while True:
    user_input = input("Ask a question (or type 'exit'): ").strip()
    if user_input.lower() in ['exit', 'quit']:
        break

    try:
        # Store user input in LangChain memory
        memory.chat_memory.add_user_message(user_input)

        # Send input to Gemini
        response = chat.send_message(user_input)
        print("\n🔍 Answer:", response.text, "\n")

        # Store bot response in memory
        memory.chat_memory.add_ai_message(response.text)

    except Exception as e:
        print(f"❌ Error: {e}")


Ask a question (or type 'exit'): exit
